In [1]:
#load packages
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


/work/MarkusLundsfrydJensen#1865/miniconda3/envs/blame_bert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#next step -> implement batching

In [2]:
#OBS need to be made usable
class BlameDetectorDa(object):

    def __init__(self, model_path, max_length = 512):

        self.model_path = model_path
        self.max_length = max_length
        #self.batch_size = batch_size

        self.model_initialization()

        return

    def model_initialization(self):
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_path,
            device_map = 'auto'
        )

        self.model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        
            
        # Move to GPU if available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model.to(self.device)

        print(f"Model loaded successfully on {self.device}")

        return

    def predict(self):
        """Make a prediction on a single text input."""
        # Tokenize input
        inputs = self.tokenizer(
            self.text,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
       
        # Move inputs to device
        inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
        # Make prediction
        with torch.no_grad():
            outputs = self.model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0][predicted_class].item()
        
        return predicted_class, confidence, probabilities[0].cpu().numpy()

    def run_prediction(self, text):

        self.text = text
        predicted_class, confidence, probs = self.predict()
            
        return predicted_class, confidence



In [3]:
BD = BlameDetectorDa(model_path = "/work/MarkusLundsfrydJensen#1865/test_models_save/Option_2")

2025-10-29 14:23:22.938561: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-29 14:23:29.108378: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 14:24:02.047962: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Model loaded successfully on cuda


In [4]:
import json
json_path = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/Model_data/validation_set.json"

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

for entry in data:
    text = entry["text"]

    entry["pred"], _ = BD.run_prediction(text)

    

true_labels = 0
pred_true = 0
false_labels = 0
pred_false = 0

correct_pred = 0
incorrect_pred = 0

for entry in data:
    if entry["label"] == 1:
        true_labels += 1
    if entry["pred"] == 1:
        pred_true +=1

    if entry["label"] == 0:
        false_labels += 1
    if entry["pred"] == 0:
        pred_false +=1

    if entry["label"] == entry["pred"]:
        correct_pred +=1

    if entry["label"] != entry["pred"]:
        incorrect_pred +=1


print(correct_pred/len(data))

/work/MarkusLundsfrydJensen#1865/miniconda3/envs/blame_bert/lib/python3.12/site-packages/torch/backends/cuda/__init__.py:131: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  return torch._C._get_cublas_allow_tf32()
/work/MarkusLundsfrydJensen#1865/miniconda3/envs/blame_bert/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:312: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for bett

0.7635658914728682


In [6]:
pred_true

129